In [1]:
# Scaled variable
import pyvista
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np

In [2]:
#-------------------------------------------------------------------------%
#                Constantes de l'ingenieur homogénéisé
#-------------------------------------------------------------------------%
E1 = 181e9
E2 = 10.3e9
nu12 = 0.28
G12 = 7.17e9
eta121 = 0
eta122 = 0

# Epaisseur pli
hel = 1e-3

# Sequence
seq = np.array([22.5, -67.5, -67.5, 22.5, -22.5, 67.5, 67.5, -22.5]) * np.pi / 180

# Nombre total N de couches
N = len(seq)

# Epaisseur totale
htot = N * hel

# Tenseur de souplesse
S = np.zeros((3, 3))
S[0, 0] = 1 / E1
S[1, 1] = 1 / E2
S[2, 2] = 1 / G12
S[0, 1] = -nu12 / E1
S[0, 2] = eta121 * S[0, 0]
S[1, 2] = eta122 * S[1, 1]
S[1, 0] = S[0, 1]
S[2, 0] = S[0, 2]
S[2, 1] = S[1, 2]

# Paramètres polaires du tenseur S
t0 = (S[0, 0] - 2 * S[0, 1] + S[2, 2] + S[1, 1]) / 8
t1 = (S[0, 0] + 2 * S[0, 1] + S[1, 1]) / 8

a0 = S[0, 0] - 2 * S[0, 1] - S[2, 2] + S[1, 1]
b0 = 2 * (S[0, 2] - S[1, 2])
r0 = np.sqrt(a0**2 + b0**2) / 8
phi0 = np.arctan2(b0, a0) / 4

a1 = S[0, 0] - S[1, 1]
b1 = S[0, 2] + S[1, 2]
r1 = np.sqrt(a1**2 + b1**2) / 8
phi1 = np.arctan2(b1, a1) / 2

# Angle polaire
d = np.arange(0.01, 2 * np.pi, np.pi / 180)

# Tenseur de souplesse polaire
Sxx = t0 + 2 * t1 + r0 * np.cos(4 * (phi0 - d)) + 4 * r1 * np.cos(2 * (phi1 - d))
Syy = t0 + 2 * t1 + r0 * np.cos(4 * (phi0 - d)) - 4 * r1 * np.cos(2 * (phi1 - d))
Sss = 4 * (t0 - r0 * np.cos(4 * (phi0 - d)))
Sxy = -t0 + 2 * t1 - r0 * np.cos(4 * (phi0 - d))
Sxs = 2 * (r0 * np.sin(4 * (phi0 - d)) + 2 * r1 * np.sin(2 * (phi1 - d)))
Sys = 2 * (-r0 * np.sin(4 * (phi0 - d)) + 2 * r1 * np.sin(2 * (phi1 - d)))

theta = np.arange(0, 2 * np.pi, np.pi / 180)

## a etoile
t0ax = t0
t1ax = t1

re0a = 0
im0a = 0
re1a = 0
im1a = 0

for k in range(-N//2, N//2 + 1):
    if N % 2 == 0 and k == N//2:
        continue
    l = k + N//2
    c4 = np.cos(4 * seq[l])
    s4 = np.sin(4 * seq[l])
    c2 = np.cos(2 * seq[l])
    s2 = np.sin(2 * seq[l])
    re0a += (1. / N) * r0 * (np.cos(4 * phi0) * c4 - np.sin(4 * phi0) * s4)
    im0a += (1. / N) * r0 * (np.cos(4 * phi0) * s4 + np.sin(4 * phi0) * c4)
    re1a += (1. / N) * r1 * (np.cos(2 * phi1) * c2 - np.sin(2 * phi1) * s2)
    im1a += (1. / N) * r1 * (np.cos(2 * phi1) * s2 + np.sin(2 * phi1) * c2)

r0ax = np.sqrt(re0a**2 + im0a**2)
phi0ax = np.arctan2(im0a, re0a) / 4

r1ax = np.sqrt(re1a**2 + im1a**2)
phi1ax = np.arctan2(im1a, re1a) / 2

## d etoile
dk = np.zeros(N)

t0dx = t0
t1dx = t1

re0d = 0
im0d = 0
re1d = 0
im1d = 0

for k in range(-N//2, N//2 ):
    if N % 2 == 0:
        if k >= 0:
            l = k + N//2
        else:
            l = k + N//2 + 1
        if k == 0:
            dk = 0
        else:
            dk = 12 * (k)**2 - 12 * abs(k) + 4  # k=2p paire
    else:
        l = k + N//2 + 1
        dk = 12 * k**2 + 1  # k=2p+1 impaire

    c4 = np.cos(4 * seq[l])
    s4 = np.sin(4 * seq[l])
    c2 = np.cos(2 * seq[l])
    s2 = np.sin(2 * seq[l])

    re0d += (1. / N**3) * dk * r0 * (np.cos(4 * phi0) * c4 - np.sin(4 * phi0) * s4)
    im0d += (1. / N**3) * dk * r0 * (np.cos(4 * phi0) * s4 + np.sin(4 * phi0) * c4)
    re1d += (1. / N**3) * dk * r1 * (np.cos(2 * phi1) * c2 - np.sin(2 * phi1) * s2)
    im1d += (1. / N**3) * dk * r1 * (np.cos(2 * phi1) * s2 + np.sin(2 * phi1) * c2)

r0dx = np.sqrt(re0d**2 + im0d**2)
phi0dx = np.arctan2(im0d, re0d) / 4

r1dx = np.sqrt(re1d**2 + im1d**2)
phi1dx = np.arctan2(im1d, re1d) / 2

# Module de young Ex en dehors des axes d'orthotropie
axx = t0ax + 2 * t1ax + r0ax * np.cos(4 * (phi0ax - theta)) + 4 * r1ax * np.cos(2 * (phi1ax - theta))
ayy = t0ax + 2 * t1ax + r0ax * np.cos(4 * (phi0ax - theta)) - 4 * r1ax * np.cos(2 * (phi1ax - theta))
dxx = t0dx + 2 * t1dx + r0dx * np.cos(4 * (phi0dx - theta)) + 4 * r1dx * np.cos(2 * (phi1dx - theta))
dyy = t0dx + 2 * t1dx + r0dx * np.cos(4 * (phi0dx - theta)) - 4 * r1dx * np.cos(2 * (phi1dx - theta))
axy = -t0ax + 2 * t1ax - r0ax * np.cos(4 * (phi0ax - theta))
ayx = -t0ax + 2 * t1ax - r0ax * np.cos(4 * (phi0ax - theta))

# Module de cisaillement Gx en dehors des axes d'orthotropie (pourquoi x4?)
ass = 4 * (t0ax - r0ax * np.cos(4 * (phi0ax - theta)))
dss = 4 * (t0dx - r0dx * np.cos(4 * (phi0dx - theta)))

# Méthode pour avoir le monocouche
E1hom = 1. / axx[0]
E2hom = 1. / axx[90]
G12hom = 1. / ass[0]
nu12hom = -E1hom * axy[0]
eta121hom = 0
eta122hom = 0

# Tenseur de souplesse monocouche
Shom = np.zeros((3, 3))
Shom[0, 0] = 1 / E1hom
Shom[1, 1] = 1 / E2hom
Shom[2, 2] = 1 / G12hom
Shom[0, 1] = -nu12hom / E1hom
Shom[0, 2] = eta121hom * S[0, 0]
Shom[1, 2] = eta122hom * S[1, 1]
Shom[1, 0] = Shom[0, 1]
Shom[2, 0] = Shom[0, 2]
Shom[2, 1] = Shom[1, 2]

Chom = np.linalg.inv(Shom)

# Affichage des résultats
print("E1hom =", E1hom)
print("E2hom =", E2hom)
print("G12hom =", G12hom)
print("nu12hom =", nu12hom)
print("Chom =", Chom)

E1hom = 18009385971.160084
E2hom = 18009385971.160084
G12hom = 8157399399.518665
nu12hom = 0.10386810116363444
Chom = [[1.82058008e+10 1.89100195e+09 0.00000000e+00]
 [1.89100195e+09 1.82058008e+10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.15739940e+09]]


In [3]:
# Définir la matrice de rigidité comme un tenseur de quatrième ordre
C_ijkl = np.zeros((2, 2, 2, 2))
C_ijkl[0, 0, 0, 0] = Chom[0, 0]
C_ijkl[0, 0, 1, 1] = Chom[0, 1]
C_ijkl[1, 1, 0, 0] = Chom[1, 0]
C_ijkl[1, 1, 1, 1] = Chom[1, 1]
C_ijkl[0, 1, 0, 1] = Chom[2, 2]
C_ijkl[1, 0, 1, 0] = Chom[2, 2]
print(C_ijkl)

[[[[1.82058008e+10 0.00000000e+00]
   [0.00000000e+00 1.89100195e+09]]

  [[0.00000000e+00 8.15739940e+09]
   [0.00000000e+00 0.00000000e+00]]]


 [[[0.00000000e+00 0.00000000e+00]
   [8.15739940e+09 0.00000000e+00]]

  [[1.89100195e+09 0.00000000e+00]
   [0.00000000e+00 1.82058008e+10]]]]


In [4]:
L = 1
W = 0.2

domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([0, 0]), np.array([L, W])],
                          [20, 4], cell_type=mesh.CellType.quadrilateral)

In [5]:
# Définir les fonctions de forme et les fonctions test
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))
# Afficher la taille du bloc de l'espace fonctionnel
block_size = V.dofmap.index_map_bs
print(f"Taille du bloc de l'espace fonctionnel V: {block_size}")
print(f"Dimension de la géométrie: {domain.geometry.dim}")

Taille du bloc de l'espace fonctionnel V: 2
Dimension de la géométrie: 2


In [6]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Définir la déformation
epsilon = ufl.sym(ufl.grad(u))

# Définir la contrainte
sigma = ufl.as_tensor([[sum(C_ijkl[i, j, k, l] * epsilon[k, l] for k in range(2) for l in range(2)) for j in range(2)] for i in range(2)])

In [7]:
print(u)
print(v)
print(epsilon)
print(sigma)

v_1
v_0
sym(grad(v_1))
[
  [18205800753.463 * (sym(grad(v_1)))[0, 0] + 1891001954.4256673 * (sym(grad(v_1)))[1, 1], 8157399399.518665 * (sym(grad(v_1)))[0, 1]],
  [8157399399.518665 * (sym(grad(v_1)))[1, 0], 1891001954.4256673 * (sym(grad(v_1)))[0, 0] + 18205800753.463 * (sym(grad(v_1)))[1, 1]]
]


In [8]:
# Localisation de la région encastrée
def clamped_boundary(x):
    return x[0] <= 0.1
    
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)

u_D = np.array([0, 0], dtype=default_scalar_type)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [9]:
print(fdim)
print(boundary_facets)
print(u_D)
print(bc)

1
[ 0  1  3  5  9 17 26 27]
[0. 0.]


In [10]:
T = fem.Constant(domain, default_scalar_type((-10000000, 0)))

In [11]:
print(T)

c_0


In [12]:
ds = ufl.Measure("ds", domain=domain)

In [13]:
print(ds)

ds(subdomain_id=everywhere, domain=<Mesh #0>)


In [14]:
# Force volumique (ici nulle)
f = fem.Constant(domain, default_scalar_type((0, 0)))
# Définir la forme bilinéaire pour le problème de l'élasticité
a = ufl.inner(sigma, ufl.sym(ufl.grad(v))) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

In [19]:
len(f)

2

In [16]:
# Définir le problème linéaire
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
# Résoudre le problème
uh = problem.solve()

In [18]:
import numpy
import pyvista as pv
# Création du maillage pour PyVista basé sur les coordonnées des dofs
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)

# Créez la grille PyVista et ajoutez les valeurs des dofs à la grille
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

# Adapter les vecteurs de déplacement en 3D pour PyVista (en ajoutant une troisième dimension de zéros)
uh_vectors = np.zeros((u_geometry.shape[0], 3))
uh_vectors[:, :2] = uh.x.array.reshape(-1, 2)

# Ajouter les vecteurs de déplacement à la grille
u_grid.point_data["u"] = uh_vectors

# Visualisation
p = pv.Plotter()

u_grid.point_data["u"] = uh.x.array.reshape((u_geometry.shape[0], 2))
warped = u_grid.warp_by_scalar("u", factor=25)
p.add_mesh(warped, show_edges=True, scalar_bar_args={
    "title": "u",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Pâle Winckler", font_size=12, color="black", position="upper_edge")
p.show_bounds(color="black")
p.add_axes(color="black")
p.set_background("grey")
p.show()

Widget(value='<iframe src="http://localhost:64737/index.html?ui=P_0x149d241a0_1&reconnect=auto" class="pyvista…